In [11]:
import pandas as pd
import numpy as np
import datetime

# Retrieve River Flows
flow = pd.read_csv('C:/Users/Scott/Desktop/Projects/River_Data/Ingestion/CurrentWater.csv')
# Retrieve Precip
rain = pd.read_csv('C:/Users/Scott/Desktop/Projects/River_Data/Ingestion/CurrentPrecip.csv',index_col=0)
# Retrieve Forecast
rainforecast = pd.read_csv('C:/Users/Scott/Desktop/Projects/River_Data/Ingestion/NewWeatherForecast.csv',index_col=0)
# Retrieve River References
reference = pd.read_csv('C:/Users/Scott/Desktop/Projects/River_Data/RiverReferenceTable.csv')

rainforecast['EndDate'] = pd.to_datetime(rainforecast.endTime).dt.date
rainforecast['StartDate'] = pd.to_datetime(rainforecast.startTime).dt.date

# Join on Date
flow = flow.set_index('Name').join(reference[['USGS Name','Name']].set_index('USGS Name')).reset_index()
flow['CFS'] = flow.Value
flow['DateTime'] = pd.to_datetime(flow.DateTime)
flow['Date'] = flow.DateTime.dt.date
flow = flow[['Date','Name','CFS']].groupby(['Date','Name']).mean().reset_index()
flow['Date_Name'] = flow.Date.astype(str)+'_'+flow.Name


rain = rain.drop_duplicates()
rain['Date'] = pd.to_datetime(rain.Date).dt.date
rain['Date_Name'] = rain.Date.astype('str')+'_'+rain.Name
rain = rain[['Date_Name','Date','Name','Precip']].groupby(['Date','Name','Date_Name']).mean().reset_index()

df = flow.set_index('Date_Name').join(rain[['Date_Name','Precip']].set_index('Date_Name'))
RiverList = df.Name.unique()
full = df.loc[df.Precip.isnull()==False]

# Lag Precip and Forecast data by correlation strength to river flow
RiverData = pd.DataFrame()
for n in RiverList:
    s = 0
    CorrList = list()
    while s < 10:
        d = full[['CFS','Precip']].loc[full.Name==n]
        d['Precip'] = d['Precip'].shift(s)
        r = d.dropna()
        corr = r.Precip.corr(r.CFS)
        CorrList.append([s,corr])
        s+=1
    Corrs = pd.DataFrame(CorrList,columns=['PrecipLag','Corr'])
    BestShift = Corrs.sort_values(by='Corr',ascending=False).head(1)
    BestShift['Name'] = n
    RiverData = RiverData.append(BestShift)

flow = flow.loc[flow.Date>=datetime.date(2022,12,1)]
rain = rain.loc[rain.Date>=datetime.date(2022,12,1)]
rainforecast = rainforecast.loc[rainforecast.EndDate>=datetime.date(2022,12,1)]

c:\Users\Scott\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2683: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
c:\Users\Scott\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2542: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
